In [77]:
!pip install momentfm

In [78]:
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torch
from momentfm import MOMENTPipeline

import torch.optim as optim
from torch import nn
from torchmetrics import MeanSquaredError, MeanAbsoluteError

In [79]:
# Load data
df = pd.read_csv("../data/preprocessed_hourly_data.csv", parse_dates=['datetime'])
df = df.sort_values('datetime').reset_index(drop=True)

# Select features and target
features = ['Open', 'High', 'Low', 'Close', 'SMA_200', 'ATR_168']
target = 'Close'
data = df[features].values.astype(np.float32)

print(df.shape)
df.head()

(116504, 8)


,datetime,Timestamp,Open,High,Low,Close,SMA_200,ATR_168
0,2012-01-09 17:00:00,1.326128e+09,6.9,6.9,6.5,6.5,5.83495,0.032560
1,2012-01-09 18:00:00,1.326132e+09,6.5,6.6,6.5,6.5,5.84455,0.033155
2,2012-01-09 19:00:00,1.326136e+09,6.5,6.6,6.5,6.6,5.85465,0.033750
3,2012-01-09 20:00:00,1.326139e+09,6.6,6.6,6.6,6.6,5.86475,0.033750
4,2012-01-09 21:00:00,1.326143e+09,6.6,6.6,6.6,6.6,5.87485,0.033750


In [80]:
# Normalize features channel-wise (fit on train portion only)
n_total = len(data)
split_idx = int(0.8 * n_total)

# Create sliding windows
input_len = 512
horizon = 360 # forecast horizon, 360 hours = 15 days

windows, targets = [], []
for i in range(n_total - input_len - horizon + 1):
    inp = data[i:i+input_len, :].T  # shape: (channels, 512)
    tgt = data[i+input_len:i+input_len+horizon, features.index(target)]  # Close prices
    windows.append(inp)
    targets.append(tgt)
print(inp.shape, tgt.shape)

windows = np.stack(windows)   # shape: (N_windows, channels, input_len=512)
targets = np.stack(targets)   # shape: (N_windows, horizon=360)

print(windows.shape, targets.shape)

(6, 512) (360,)
(115633, 6, 512) (115633, 360)


In [81]:
# Split into train/val/test (chronological)
n_windows = len(windows)
print(f"Total number of windows: {n_windows}")

train_end = int(0.8 * n_windows)
val_end = int(0.9 * n_windows)
train_X, val_X, test_X = windows[:train_end], windows[train_end:val_end], windows[val_end:]
train_y, val_y, test_y = targets[:train_end], targets[train_end:val_end], targets[val_end:]

Total number of windows: 115633


In [82]:
class ForecastDataset(Dataset):
    def __init__(self, X, Y):
        self.X = torch.from_numpy(X).float()  # shape (N, C, L): (N_windows, channels=6, input_len=512)
        self.Y = torch.from_numpy(Y).float()  # shape (N, H): (N_windows, horizon=360)
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        timeseries = self.X[idx]              # (C, L)
        # print(timeseries.shape)
        forecast = self.Y[idx]               # (H,)
        input_mask = torch.ones(timeseries.shape[-1])  # (L,), all observed
        # print(input_mask.shape)
        return timeseries, forecast, input_mask


In [88]:
train_ds = ForecastDataset(train_X, train_y)
val_ds   = ForecastDataset(val_X,   val_y)
test_ds  = ForecastDataset(test_X,  test_y)

train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)
val_loader   = DataLoader(val_ds,   batch_size=16, shuffle=False)
test_loader  = DataLoader(test_ds,  batch_size=16, shuffle=False)

In [84]:

# Load MOMENT pipeline for forecasting
model = MOMENTPipeline.from_pretrained(
    "AutonLab/MOMENT-1-small",
    model_kwargs={
        'task_name': 'forecasting',
        'forecast_horizon': horizon,
        # 'forecast_dim': 1,
        'head_dropout': 0.1,
        'weight_decay': 0.0,
        'freeze_encoder': True,
        'freeze_embedder': True,
        'freeze_head': False
    }
)
model.init()
# model = moment_pipe.model  # The PyTorch model with forecasting head
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

c:\Users\4310129\AppData\Local\anaconda3\envs\AI417-DL\lib\site-packages\momentfm\models\moment.py:174: UserWarning: Only reconstruction head is pre-trained. Classification and forecasting heads must be fine-tuned.
  warnings.warn("Only reconstruction head is pre-trained. Classification and forecasting heads must be fine-tuned.")


MOMENTPipeline(
  (normalizer): RevIN()
  (tokenizer): Patching()
  (patch_embedding): PatchEmbedding(
    (value_embedding): Linear(in_features=8, out_features=512, bias=False)
    (position_embedding): PositionalEmbedding()
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (

In [90]:
num_epochs = 50

# Loss functions
mse_loss_fn = nn.MSELoss()
huber_loss_fn = nn.SmoothL1Loss()
mse_loss_fn.to(device)
huber_loss_fn.to(device)

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-2)

# Metrics
metric_mse = MeanSquaredError().to(device)
metric_mae = MeanAbsoluteError().to(device)

# Scheduler
from torch.optim.lr_scheduler import OneCycleLR
total_steps = len(train_loader) * num_epochs
scheduler = OneCycleLR(optimizer, max_lr=1e-4, total_steps=total_steps)
max_grad_norm = 5.0


In [ ]:
# Training loop
for epoch in range(num_epochs):
    model.train()
    train_losses = []
    for series, target, mask in train_loader:
        series = series.to(device)     # (batch, L, C)
        target = target.to(device)     # (batch, H)
        mask   = mask.to(device)       # (batch, L)

        optimizer.zero_grad()
        output = model(x_enc=series, input_mask=mask)  # forward pass # shape [B, 360] from forecasting head
        # print(output.forecast.shape, target.shape)
        # loss = mse_loss_fn(output.forecast, target)
        loss = huber_loss_fn(output.forecast[:, 3, :], target)
        loss.backward()

        # torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        # scheduler.step()
        
        train_losses.append(loss.item())
    avg_train_loss = np.mean(train_losses)

    model.eval()
    val_losses = []
    metric_mse.reset(); metric_mae.reset()
    with torch.no_grad():
        for series, target, mask in val_loader:
            series = series.to(device)     # (batch, L, C)
            target = target.to(device)     # (batch, H)
            mask   = mask.to(device)       # (batch, L)

            output = model(x_enc=series, input_mask=mask)
            val_loss = huber_loss_fn(output.forecast[:, 3, :], target)
            val_losses.append(val_loss.item())
            preds = output.forecast[:, 3, :].contiguous()
            metric_mse(preds, target)
            metric_mae(preds, target)
    avg_val_loss = np.mean(val_losses)
    val_mse = metric_mse.compute().item()
    val_mae = metric_mae.compute().item()

    print(f"Epoch {epoch+1} | Train Loss {avg_train_loss:.4f} | Val Loss {avg_val_loss:.4f}  | Val MSE {val_mse:.4f}, Val MAE {val_mae:.4f}")

c:\Users\4310129\AppData\Local\anaconda3\envs\AI417-DL\lib\site-packages\torch\_dynamo\eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
c:\Users\4310129\AppData\Local\anaconda3\envs\AI417-DL\lib\site-packages\torch\utils\checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


In [ ]:
model.eval()
test_losses = []
metric_mse.reset(); metric_mae.reset()
with torch.no_grad():
    for series, target, mask in test_loader:
        series, target, mask = series.to(device), target.to(device), mask.to(device)
        output = model(x_enc=series, input_mask=mask)

        preds = output.forecast[:, 3, :].contiguous()
        metric_mse(preds, target)
        metric_mae(preds, target)

        # For reporting Huber:
        test_losses.append(huber_loss_fn(output.forecast[:, 3, :], target).item())
test_mse = metric_mse.compute().item()
test_mae = metric_mae.compute().item()
test_huber = np.mean(test_losses)

print(f"Test MSE: {test_mse:.4f}, Test MAE: {test_mae:.4f}, Test Huber: {test_huber:.4f}")


Test MSE: 25419528.0000, Test MAE: 3614.6968, Test Huber: 3615.6266
